<a href="https://colab.research.google.com/github/CIGLR-ai-lab/GreatLakes-TempSensors/blob/main/notebooks/debugging/04_dcj_active_learning_append_error_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install cartopy deepsensor rioxarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.3/91.3 kB 13.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.2/210.2 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting re

In [1]:
# Mounting Google Drive in Google Colab
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import logging
logging.captureWarnings(True)

import deepsensor.torch
from deepsensor.model import ConvNP
from deepsensor.data import DataProcessor, TaskLoader
from deepsensor.train import set_gpu_default_device
from deepsensor.train import Trainer
from deepsensor.active_learning import GreedyAlgorithm
from deepsensor.active_learning.acquisition_fns import Stddev
import pandas as pd
import xarray as xr
import numpy as np
from tqdm import notebook

# Load datasets
dat14 = '/content/gdrive/MyDrive/CIGLR Summer Fellows/DeepSensor-2024/sst-input-data-sample/GLSEA3_NETCDF/GLSEA3_2014.nc'

#dat = xr.open_mfdataset([dat14],
#                        concat_dim='time',
#                        combine='nested',
#                        chunks={'lat': 'auto', 'lon': 'auto'})
dat = xr.open_dataset(dat14)

# Process data (single year, so no need to change to climatology and anomalies)
mdat = dat.where(np.isnan(dat.sst) == False, -0.009)
#climatology = mdat.groupby('time.dayofyear').mean('time')
#anomalies = mdat.groupby('time.dayofyear') - climatology

data_processor = DataProcessor(x1_name="lat", x2_name="lon")
mdat_ds = data_processor(mdat)

task_loader = TaskLoader(
    context = mdat_ds,
    target = mdat_ds
)

val_tasks = []
for date in pd.date_range('2014-01-01T12:00:00.000000000', '2014-12-31T12:00:00.000000000'):
    N_context = np.random.randint(0, 100)
    task = task_loader(date, context_sampling="all", target_sampling="all")
    val_tasks.append(task)

model = ConvNP(data_processor, task_loader)

def compute_val_rmse(model, val_tasks):
    errors = []
    target_var_ID = task_loader.target_var_IDs[0][0]  # assume 1st target set and 1D
    for task in np.random.choice(val_tasks, 50, replace = False):
        mean = data_processor.map_array(model.mean(task), target_var_ID, unnorm=True)
        true = data_processor.map_array(task["Y_t"][0], target_var_ID, unnorm=True)
        errors.extend(np.abs(mean - true))
    return np.sqrt(np.mean(np.concatenate(errors) ** 2))

def gen_tasks(dates, progress=True):
    tasks = []
    for date in notebook.tqdm(dates, disable=not progress):
        task = task_loader(date, context_sampling=["all"], target_sampling="all")
        tasks.append(task)
    return tasks

set_gpu_default_device()
losses = []
val_rmses = []
train_range = pd.date_range('2014-01-02T12:00:00.000000000', '2014-08-31T12:00:00.000000000')
val_range = pd.date_range('2014-09-01T12:00:00.000000000', '2014-12-31T12:00:00.000000000')
val_rmse_best = np.inf
trainer = Trainer(model, lr=5e-5)
for epoch in range(5):
    train_tasks = gen_tasks(pd.date_range(train_range[0], train_range[1])[::5], progress=False)
    batch_losses = trainer(train_tasks)
    losses.append(np.mean(batch_losses))
    val_rmses.append(compute_val_rmse(model, val_tasks))
    if val_rmses[-1] < val_rmse_best:
        val_rmse_best = val_rmses[-1]

  coefs = mat.inv()[:, deriv] * np.math.factorial(deriv)

  / np.math.factorial(order)

  coefs = mat.inv()[:, deriv] * np.math.factorial(deriv)

  / np.math.factorial(order)



In [ ]:
mdat

In [ ]:
alg = GreedyAlgorithm(
    model,
    X_s = mdat,
    X_t = mdat,
    context_set_idx=0,
    target_set_idx=0,
    N_new_context=3,
    progress_bar=True,
)

acquisition_fn = Stddev(model)

val_dates = pd.date_range('2014-09-01T12:00:00.000000000', '2014-12-31T12:00:00.000000000')[::5]
placement_dates = val_dates
placement_tasks = task_loader(placement_dates, context_sampling="all")

# Trigger the error
X_new_df, acquisition_fn_ds = alg(acquisition_fn, placement_tasks)

# Error: GriddedDataError: Cannot append to gridded data

# Debug notes

This notebook runs without code errors on Google Colab with a T4 GPU, but the sessions have been crashing after using all available RAM. (This is 15 GB GPU RAM and 12.7 GB CPU/System RAM). Perhaps it could be run with additional GPUs on U-M HPC?